# Imports 

In [1]:
import pandas as pd
import re 
import time
import clipboard
import pyautogui
import datetime

## Initializing the date

In [10]:
today=datetime.datetime.today()

In [11]:
today = str(datetime.datetime.date(today))

In [12]:
today

'2023-10-15'

## Inventory 
### Make Sure to grab 40 Gold Link when Applicable 

In [3]:
fidi = {
    '90 Washington':'https://streeteasy.com/for-rent/nyc/building:109',
    '100 John': 'https://streeteasy.com/for-rent/nyc/building:362',
    '37 Wall': 'https://streeteasy.com/for-rent/nyc/building:150',
    '75 West': 'https://streeteasy.com/for-rent/nyc/building:282',
    '116 John': 'https://streeteasy.com/for-rent/nyc/building:356',
    '85 John ': 'https://streeteasy.com/for-rent/nyc/building:416',
    '11 Maiden': 'https://streeteasy.com/for-rent/nyc/building:324',
    '63 Wall': 'https://streeteasy.com/for-rent/nyc/building:153',
    '67 Wall': 'https://streeteasy.com/for-rent/nyc/building:154',
    '90 West': 'https://streeteasy.com/for-rent/nyc/building:285',
    '15 Park Row': 'https://streeteasy.com/for-rent/nyc/building:497',
    '100 Maiden Lane': 'https://streeteasy.com/for-rent/nyc/building:232',
    '95 Wall': 'https://streeteasy.com/for-rent/nyc/building:198',
    '10 Hanover': 'https://streeteasy.com/for-rent/nyc/building:192',
    '2 Gold': 'https://streeteasy.com/for-rent/nyc/building:352',
    '45 Wall': 'https://streeteasy.com/for-rent/nyc/building:151',
    '71 Broadway': 'https://streeteasy.com/for-rent/nyc/building:129',
    '84 William': 'https://streeteasy.com/for-rent/nyc/building:340',
    '180 Water': 'https://streeteasy.com/for-rent/nyc/building:365',
    '20 Broad': 'https://streeteasy.com/for-rent/nyc/building:137',
    '20 Exchnage': 'https://streeteasy.com/for-rent/nyc/building:155',
    '21 West': 'https://streeteasy.com/for-rent/nyc/building:60',
    '70 Pine': 'https://streeteasy.com/for-rent/nyc/building:225',
    '29 Cliff': 'https://streeteasy.com/for-rent/nyc/building:406',
#     '40 Gold':  'https://streeteasy.com/building/40-gold-street-new_york',
    '254 Front': 'https://streeteasy.com/for-rent/nyc/building:605',
    '15 Cliff': 'https://streeteasy.com/for-rent/nyc/building:409',
    '33 Gold': 'https://streeteasy.com/for-rent/nyc/building:424',
    '108 Fulton': 'https://streeteasy.com/for-rent/nyc/building:429',
    '110 Greenwhich': 'https://streeteasy.com/for-rent/nyc/building:272',
    '8 Spruce': 'https://streeteasy.com/for-rent/nyc/building:582',
    '19 Dutch': 'https://streeteasy.com/for-rent/nyc/building:438',
    '7 Dey': 'https://streeteasy.com/for-rent/nyc/building:6637473',
    '2 Water': 'https://streeteasy.com/for-rent/nyc/building:25', 
    '200 Water': 'https://streeteasy.com/for-rent/nyc/building:392',
    '1 West':'https://streeteasy.com/for-rent/nyc/building:61'
}

### Functions Needed for parsing the scrape

In [13]:
def extract_with_first_method(text):
    listings = re.split(r'\n\d+\s', text)
    results = []

    for listing in listings[1:]:  # Skip the first empty element
        listing_dict = {}

        # Extract the listing title and price
        title_price_match = re.match(r'(.+)\n\$(\d+(?:,\d+)?)\s', listing)
        if title_price_match:
            listing_dict['Title'] = title_price_match.group(1).strip()
            listing_dict['Price'] = int(title_price_match.group(2).replace(',', ''))

        # Extract other details using regular expressions
        other_details = re.findall(r'(Studio|\d+\sBed)\s(.+?)\sBath', listing)
        if other_details:
            beds, bath = other_details[0]
            listing_dict['Bedrooms'] = beds
            listing_dict['Bathrooms'] = bath

        if all(key in listing_dict for key in ['Title', 'Price', 'Bedrooms', 'Bathrooms']):
            results.append(listing_dict)

    return results
def extract_with_second_method(text):
    sections = re.split(r'\n(?=\d+\s[A-Za-z0-9\s#]+)', text)
    results = []

    # Define regular expressions to extract specific details
    patterns = {
        'unit': re.compile(r'(\d+\s[A-Za-z0-9\s#]+)'),
        'price': re.compile(r'(\$[\d,]+(?:\.\d{2})?)'),
        'beds': re.compile(r'(\d+(?:\.\d+)?)\sBeds'),
        'baths': re.compile(r'(\d+(?:\.\d+)?)\sBath'),
    }

    for section in sections:
        listing_dict = {}
        for key, pattern in patterns.items():
            match = pattern.search(section)
            if match:
                listing_dict[key] = match.group(1).strip()

        if all(key in listing_dict for key in ['unit', 'price', 'beds', 'baths']):
            results.append({
                'Title': listing_dict['unit'],
                'Price': listing_dict['price'],
                'Bedrooms': listing_dict['beds'],
                'Bathrooms': listing_dict['baths'],
            })

    return results

### Collecting the data 

In [133]:
master_df = []

for building_name, link in fidi.items():
    current_page = 1

    while True:
        if current_page == 1:
            page_url = link  # Use the base link for the first iteration
        else:
            page_url = f"{link}?page={current_page}"  # Append ?page=X for subsequent iterations

        time.sleep(1.5)
        pyautogui.hotkey('command', 't')  # Open a new tab
        time.sleep(1.5)
        pyautogui.hotkey('command', 'l')
        pyautogui.hotkey('command', 'l')
        pyautogui.press('backspace')
        time.sleep(1)
        # Simulate typing the URL and pressing Enter
        pyautogui.write(page_url)
        pyautogui.press('enter')
        time.sleep(4)
        pyautogui.hotkey('command', 'a')
        pyautogui.hotkey('command', 'c')# Wait for the page to load

        clipboard_content = clipboard.paste()
        text = clipboard_content

        listing_data = extract_with_first_method(text)
        df = pd.DataFrame(listing_data)


        # Extract the second method results
        second_method_results = extract_with_second_method(text)
        second_df = pd.DataFrame(second_method_results)

        # Concatenate both DataFrames
        merged_df = pd.concat([df, second_df], ignore_index=True)
        merged_df
        
        if merged_df.empty:
            # If the DataFrame is empty, it means there are no more pages, so break the loop
            pyautogui.hotkey('command', 'w')
            break
        try:
            title = df['Title'].iloc[0]
            # Define a regular expression pattern to capture "Washington Street"
            pattern = r'([^#]+)'

            # Use re.search to find the pattern in the string
            match = re.search(pattern, title)

            match = match.group(1)

            filtered_df = merged_df[merged_df['Title'].apply(lambda title: match in title)]

            filtered_df
        except:
            break
        

        filtered_df['building'] = building_name  # Assign the building name to the "building" column
        master_df.append(filtered_df)
        
        pyautogui.hotkey('command', 'w')
        
        

        current_page += 1

final_df = pd.concat(master_df, ignore_index=True)
final_df['Title'] = final_df['Title'].str.replace(r'^.*?#', '', regex=True)
final_df.rename(columns={'Title': 'Unit'}, inplace=True)
final_df.rename(columns={'building': 'Building'}, inplace=True)
final_df = final_df[['Building', 'Unit', 'Price', 'Bedrooms', 'Bathrooms']]
final_df.to_csv(f'./data/fidi_{today}.csv')

/var/folders/wd/1cybbz0x4wn4b20r_6x53pt00000gn/T/ipykernel_73866/1029680070.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['building'] = building_name  # Assign the building name to the "building" column
/var/folders/wd/1cybbz0x4wn4b20r_6x53pt00000gn/T/ipykernel_73866/1029680070.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['building'] = building_name  # Assign the building name to the "building" column
/var/folders/wd/1cybbz0x4wn4b20r_6x53pt00000gn/T/ipykernel_73866/102

/var/folders/wd/1cybbz0x4wn4b20r_6x53pt00000gn/T/ipykernel_73866/1029680070.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['building'] = building_name  # Assign the building name to the "building" column
/var/folders/wd/1cybbz0x4wn4b20r_6x53pt00000gn/T/ipykernel_73866/1029680070.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['building'] = building_name  # Assign the building name to the "building" column
/var/folders/wd/1cybbz0x4wn4b20r_6x53pt00000gn/T/ipykernel_73866/102

/var/folders/wd/1cybbz0x4wn4b20r_6x53pt00000gn/T/ipykernel_73866/1029680070.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['building'] = building_name  # Assign the building name to the "building" column
/var/folders/wd/1cybbz0x4wn4b20r_6x53pt00000gn/T/ipykernel_73866/1029680070.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['building'] = building_name  # Assign the building name to the "building" column
/var/folders/wd/1cybbz0x4wn4b20r_6x53pt00000gn/T/ipykernel_73866/102

# Must Fix

### Remove listings from ads to avoid fake listings 

# Future Innovations

### 1. Create a Scraping Class and Import it so the notebook is cleaner 
### 2. Look more in depth into creating numerous virtual machines with there own GUI's and web browsers to individually collect data and optimize listing scraping. 
### 3. Run it as a .py script so it can be used directly in automation 

In [135]:
df = pd.read_csv(f'./data/fidi_{today}.csv')

In [136]:
df

,Unnamed: 0,Building,Unit,Price,Bedrooms,Bathrooms
0,0,90 Washington,6T,4190,Studio,1.0
1,1,90 Washington,17L,4300,1 Bed,1.0
2,2,90 Washington,PHF,4600,1 Bed,1.0
3,3,90 Washington,17L,4300,1 Bed,1.0
4,4,90 Washington,6T,4190,Studio,1.0
...,...,...,...,...,...,...
381,381,1 West,2322,3300,Studio,1.0
382,382,1 West,1942,3225,Studio,1.0
383,383,1 West,3604,"$7,000",2,1.0
384,384,1 West,2908,"$5,300",3,1.0


In [137]:
# Clean the Price column by removing dollar signs and commas
df['Price'] = df['Price'].str.replace('[$,]', '', regex=True).astype(float)

# Create a function to find the lowest price for each unique building and unit type
def find_lowest_prices(df):
    lowest_prices = df.groupby(['Building', 'Bedrooms']).agg({'Price': 'min'}).reset_index()
    return lowest_prices

# Find the lowest prices
lowest_prices = find_lowest_prices(df)

# Filter and extract the lowest prices for studio, 1-bedroom, and 2-bedroom units
lowest_studio = lowest_prices[lowest_prices['Bedrooms'] == 'Studio']
lowest_1_bedroom = lowest_prices[lowest_prices['Bedrooms'] == '1 Bed']
lowest_2_bedroom = lowest_prices[lowest_prices['Bedrooms'] == '2']

# Print the lowest prices for each category
print("Lowest Price Studio:")
print(lowest_studio)

print("\nLowest Price 1 Bedroom:")
print(lowest_1_bedroom)

print("\nLowest Price 2 Bedroom:")
print(lowest_2_bedroom)

Lowest Price Studio:
           Building Bedrooms   Price
3            1 West   Studio  3225.0
6        10 Hanover   Studio  3477.0
10         100 John   Studio  3500.0
13  100 Maiden Lane   Studio  3600.0
16       108 Fulton   Studio  2600.0
22         116 John   Studio  3410.0
28        180 Water   Studio  3445.0
31         19 Dutch   Studio  3976.0
35           2 Gold   Studio  3515.0
40         20 Broad   Studio  3295.0
43      20 Exchnage   Studio  3515.0
46        200 Water   Studio  4075.0
48          21 West   Studio  3562.0
52         29 Cliff   Studio  3394.0
55          33 Gold   Studio  2600.0
57          37 Wall   Studio  3250.0
60          45 Wall   Studio  3580.0
62          63 Wall   Studio  3475.0
65          67 Wall   Studio  4120.0
68            7 Dey   Studio  4925.0
71          70 Pine   Studio  3780.0
74      71 Broadway   Studio  3749.0
78         8 Spruce   Studio  3980.0
81       84 William   Studio  3400.0
84         85 John    Studio  3200.0
87    90 Washingt

In [138]:

import pandas as pd

# Assuming you have already loaded your data into a DataFrame
# Let's call it df

# Clean the Price column by removing dollar signs and commas
df['Price'] = df['Price'].replace('[$,]', '', regex=True).astype(float)

# Create a function to find the lowest price for each unique building and unit type
def find_lowest_prices(df):
    lowest_prices = df.groupby(['Building', 'Bedrooms']).agg({'Price': 'min'}).reset_index()
    return lowest_prices

# Find the lowest prices
lowest_prices = find_lowest_prices(df)

# Create a pivot table to reshape the data for all lowest prices
pivot_lowest_prices = lowest_prices.pivot(index='Building', columns='Bedrooms', values='Price').reset_index()

# Rename the columns for clarity
pivot_lowest_prices.columns.name = None  # Remove the column name
pivot_lowest_prices.rename(columns={'Studio': 'Lowest_Studio_Price', '1 Bed': 'Lowest_1_Bedroom_Price', '2': 'Lowest_2_Bedroom_Price'}, inplace=True)

# Fill any missing values with NaN (if a building doesn't have a certain unit type)
pivot_lowest_prices.fillna('', inplace=True)

# Print the DataFrame with all lowest prices
print(pivot_lowest_prices)

           Building Lowest_1_Bedroom_Price Lowest_2_Bedroom_Price       3  \
0            1 West                 4600.0                 5150.0  5300.0   
1        10 Hanover                 4816.0                 6699.0           
2          100 John                 4000.0                 4150.0  5400.0   
3   100 Maiden Lane                 4200.0                 6195.0           
4        108 Fulton                 3200.0                 3600.0           
5         11 Maiden                 3100.0                 3850.0  4995.0   
6          116 John                 4250.0                 6080.0           
7          15 Cliff                 4300.0                 5405.0           
8       15 Park Row                 3950.0                         7700.0   
9         180 Water                 4575.0                                  
10         19 Dutch                 5344.0                 6796.0           
11           2 Gold                 4270.0                 6675.0  7990.0   

In [139]:
pivot_lowest_prices[['Building', 'Lowest_1_Bedroom_Price', 'Lowest_2_Bedroom_Price', 'Lowest_Studio_Price']]

,Building,Lowest_1_Bedroom_Price,Lowest_2_Bedroom_Price,Lowest_Studio_Price
0,1 West,4600.0,5150.0,3225.0
1,10 Hanover,4816.0,6699.0,3477.0
2,100 John,4000.0,4150.0,3500.0
3,100 Maiden Lane,4200.0,6195.0,3600.0
4,108 Fulton,3200.0,3600.0,2600.0
5,11 Maiden,3100.0,3850.0,
6,116 John,4250.0,6080.0,3410.0
7,15 Cliff,4300.0,5405.0,
8,15 Park Row,3950.0,,
9,180 Water,4575.0,,3445.0


In [144]:
pivot_lowest_prices.to_csv('./data/lowest_prices.csv')